<a href="https://colab.research.google.com/github/suinkangme/comp433_project/blob/main/COMP433_Project_SK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Developing a robust CNN model to address the challenge of learning with label noise in  CIFAR10 dataset

- CIFAR10 Label : ‘airplane’, ‘automobile’, ‘bird’, ‘cat’, ‘deer’, ‘dog’, ‘frog’, ‘horse’, ‘ship’, ‘truck’.

- image size : 3x32x32




In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from torch.utils.data import DataLoader, random_split
import os
from sklearn.mixture import GaussianMixture

## Load and normalize CIFAR10

- Transform them to Tensors of normalized range [-1, 1].

In [7]:
# train dataset
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
])
cifar10_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True)

Files already downloaded and verified


In [8]:
# validation, test dataset
test_val_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
])

In [9]:
# split the dataset into train and validation
train_size = int(0.8 * len(cifar10_dataset))
val_size = len(cifar10_dataset) - train_size
train_dataset, val_dataset = random_split(cifar10_dataset, [train_size, val_size])

In [10]:
train_dataset.dataset.transform = train_transform
val_dataset.dataset.transform = test_val_transform

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

In [11]:
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=test_val_transform)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

Files already downloaded and verified


## Noise Labeling
- 5 different noise levels (10%,
30%, 50%, 80%, 90%)

In [12]:
def apply_label_noise(labels, epsilon, noise_type):
    num_labels = len(labels)
    num_flips = int(epsilon * num_labels)

    if noise_type == 'symmetric':
        # Symmetric label noise
        flip_indices = np.random.choice(num_labels, num_flips, replace=False)
        labels[flip_indices] = np.random.randint(0, 10, num_flips)
    elif noise_type == 'asymmetric':
        # Asymmetric label noise
        flip_rules = {
            9: 1,   # Truck to Automobile
            2: 0,   # Bird to Airplane
            4: 7,   # Deer to Horse
            3: 5,   # Cat to Dog
            5: 3,   # Dog to Cat
        }

        for i in range(num_labels):
            if np.random.random() < epsilon:
                labels[i] = flip_rules.get(labels[i], labels[i])

    return labels

# **Baseline Model**

## Define a  baseline CNN model

In [19]:
class BaselineModel(nn.Module):
    def __init__(self):
        super(BaselineModel, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 8, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(8, 16, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(16, 32, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        # fc layers
        self.fc_layers = nn.Sequential(
            nn.Linear(128 * 8 * 8, 120),
            nn.ReLU(inplace=True),
            nn.Linear(120, 84),
            nn.ReLU(inplace=True),
            nn.Linear(84, 10)
        )

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        x = self.fc_layers(x)
        return x

## Train & Validation and Testing - symmetric noise label

In [14]:
# noise_levels
noise_levels = [0.1, 0.3, 0.5, 0.8, 0.9]

# create a dictionary with keys in the format 'noise_level_{100 * value}'
model_dict = {f'noise_level_{int(100 * level)}_sy': None for level in noise_levels}

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# training
for epsilon in noise_levels:

    num_epochs = 20

    model = BaselineModel()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    model = model.to(device)
    model.train()

    print(f"Symmetric Training with noise level: {epsilon}")

    for epoch in range(num_epochs):
        for inputs, labels in train_loader:

            # add symmetric noise to labels
            labels = apply_label_noise(labels.numpy(), epsilon=epsilon, noise_type='symmetric')
            labels = torch.from_numpy(labels)

            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            loss.backward()
            optimizer.step()

        print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item()}')

    # validation
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            val_loss += loss.item()

            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_accuracy = correct / total
    average_val_loss = val_loss / len(val_loader)

    print(f'Validation Loss: {average_val_loss}, Validation Accuracy: {val_accuracy}')

    # save model to dictionary
    model_dict[f'noise_level_{int(100 * epsilon)}_sy'] = {
        'state_dict': model.state_dict(),
        'validation_loss': average_val_loss,
        'validation_accuracy': val_accuracy
    }

Symmetric Training with noise level: 0.1
Epoch 1/20, Loss: 1.578172206878662
Epoch 2/20, Loss: 1.0516657829284668
Epoch 3/20, Loss: 0.8020777106285095
Epoch 4/20, Loss: 1.1523828506469727
Epoch 5/20, Loss: 0.9929617643356323
Epoch 6/20, Loss: 1.02053701877594
Epoch 7/20, Loss: 1.0013731718063354
Epoch 8/20, Loss: 0.984472393989563
Epoch 9/20, Loss: 0.6997935771942139
Epoch 10/20, Loss: 0.8753437995910645
Epoch 11/20, Loss: 0.7374206781387329
Epoch 12/20, Loss: 0.9731400012969971
Epoch 13/20, Loss: 0.6782114505767822
Epoch 14/20, Loss: 0.7411990165710449
Epoch 15/20, Loss: 0.5805302262306213
Epoch 16/20, Loss: 0.5508726239204407
Epoch 17/20, Loss: 0.6559940576553345
Epoch 18/20, Loss: 0.6445692181587219
Epoch 19/20, Loss: 0.6315118074417114
Epoch 20/20, Loss: 0.5050042271614075
Validation Loss: 0.9203241023288411, Validation Accuracy: 0.7219
Symmetric Training with noise level: 0.3
Epoch 1/20, Loss: 1.8265252113342285
Epoch 2/20, Loss: 1.638409972190857
Epoch 3/20, Loss: 1.7427191734313

In [15]:
# testing
for key, model_state_info in model_dict.items():

    model = BaselineModel()
    model_state = model.state_dict()
    model_state.update({k: v for k, v in model_state_info['state_dict'].items() if k in model_state})

    # load the updated state_dict
    model.load_state_dict(model_state)

    model = model.to(device)
    model.eval()

    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = correct / total
    print(f'Test Accuracy for {key}: {accuracy}')

Test Accuracy for noise_level_10_sy: 0.7219
Test Accuracy for noise_level_30_sy: 0.7144
Test Accuracy for noise_level_50_sy: 0.7003
Test Accuracy for noise_level_80_sy: 0.5301
Test Accuracy for noise_level_90_sy: 0.1019


## Train & Validation and Testing - asymmetric noise label

In [ ]:
# noise_levels
noise_levels = [0.1, 0.3, 0.5, 0.8, 0.9]

# create a dictionary with keys in the format 'noise_level_{100 * value}'
model_dict = {f'noise_level_{int(100 * level)}_asy': None for level in noise_levels}

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# training
for epsilon in noise_levels:

    num_epochs = 20

    model = BaselineModel()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    model = model.to(device)
    model.train()

    print(f"Asymmetric Training with noise level: {epsilon}")

    for epoch in range(num_epochs):
        for inputs, labels in train_loader:

            # add symmetric noise to labels
            labels = apply_label_noise(labels.numpy(), epsilon=epsilon, noise_type='asymmetric')
            labels = torch.from_numpy(labels)

            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            loss.backward()
            optimizer.step()

        print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item()}')

    # validation
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            val_loss += loss.item()

            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_accuracy = correct / total
    average_val_loss = val_loss / len(val_loader)

    print(f'Validation Loss: {average_val_loss}, Validation Accuracy: {val_accuracy}')

    # save model to dictionary
    model_dict[f'noise_level_{int(100 * epsilon)}_asy'] = {
        'state_dict': model.state_dict(),
        'validation_loss': average_val_loss,
        'validation_accuracy': val_accuracy
    }

Asymmetric Training with noise level: 0.1
Epoch 1/5, Loss: 1.3520103693008423
Epoch 2/5, Loss: 0.9156202077865601
Epoch 3/5, Loss: 0.5246793627738953
Epoch 4/5, Loss: 0.8600916862487793
Epoch 5/5, Loss: 0.8814508318901062
Validation Loss: 0.8582057041727054, Validation Accuracy: 0.7022
Asymmetric Training with noise level: 0.3
Epoch 1/5, Loss: 1.460185170173645
Epoch 2/5, Loss: 1.3619500398635864
Epoch 3/5, Loss: 1.2083697319030762
Epoch 4/5, Loss: 1.1095333099365234
Epoch 5/5, Loss: 0.8080011010169983
Validation Loss: 0.9474637424869902, Validation Accuracy: 0.6811
Asymmetric Training with noise level: 0.5
Epoch 1/5, Loss: 1.181923270225525
Epoch 2/5, Loss: 1.2102376222610474
Epoch 3/5, Loss: 0.9125158786773682
Epoch 4/5, Loss: 0.8314177393913269
Epoch 5/5, Loss: 0.9430887699127197
Validation Loss: 1.1435436705115494, Validation Accuracy: 0.5394
Asymmetric Training with noise level: 0.8
Epoch 1/5, Loss: 1.2077199220657349
Epoch 2/5, Loss: 1.123086929321289
Epoch 3/5, Loss: 0.852846264

In [ ]:
# testing
for key, model_state_info in model_dict.items():

    model = BaselineModel()
    model_state = model.state_dict()
    model_state.update({k: v for k, v in model_state_info['state_dict'].items() if k in model_state})

    # load the updated state_dict
    model.load_state_dict(model_state)

    model = model.to(device)
    model.eval()

    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = correct / total
    print(f'Test Accuracy for {key}: {accuracy}')

Test Accuracy for noise_level_10_asy: 0.7022
Test Accuracy for noise_level_30_asy: 0.6811
Test Accuracy for noise_level_50_asy: 0.5394
Test Accuracy for noise_level_80_asy: 0.4545
Test Accuracy for noise_level_90_asy: 0.4398


# Applying dynamic weights of samples to the BaselineModel

- use compute_sample_weights(), and integrate the weighted loss calculation into the training loop


---



*hyper parameter tuning*

1.   num of epoch
2.   threshold in compute_sample_weights() : Starting with a low threshold and gradually increasing it to observe how the model's performance changes.




In [16]:
# compute sample weights based on prediction confidence
#def compute_sample_weights(outputs, labels, threshold=0.6):
#    confidences, _ = torch.max(nn.functional.softmax(outputs, dim=1), dim=1)
#    weights = torch.where(confidences < threshold, torch.tensor(1.0), torch.tensor(2.0))
#    return weights

In [23]:
# compute sample weights based on prediction confidence with threshold
def compute_sample_weights(outputs, labels, threshold=0.6):
    confidences, _ = torch.max(nn.functional.softmax(outputs, dim=1), dim=1)
    weights = torch.where(confidences < threshold, torch.tensor(1.0), 1.0 / confidences)
    return weights

## Train & Validation and Testing - symmetric noise label

In [24]:
# noise_levels
noise_levels = [0.1, 0.3, 0.5, 0.8, 0.9]

# create a dictionary with keys in the format 'noise_level_{100 * value}'
model_dict = {f'noise_level_{int(100 * level)}_sy': None for level in noise_levels}

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Training loop with dynamic sample weighting
for epsilon in noise_levels:
    num_epochs = 5

    model = BaselineModel()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    model = model.to(device)
    model.train()

    print(f"Symmetric Training with noise level: {epsilon}")

    for epoch in range(num_epochs):
        for inputs, labels in train_loader:

            # Add symmetric noise to labels
            labels = apply_label_noise(labels.numpy(), epsilon=epsilon, noise_type='symmetric')
            labels = torch.from_numpy(labels)

            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()

            outputs = model(inputs)

            # Compute sample weights using the updated function
            weights = compute_sample_weights(outputs, labels)

            # Compute weighted loss
            loss = torch.mean(weights * criterion(outputs, labels))

            loss.backward()
            optimizer.step()

        print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item()}')

    # Validation
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)

            # Use the original CrossEntropyLoss without weighting for validation
            loss = criterion(outputs, labels)

            val_loss += loss.item()

            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_accuracy = correct / total
    average_val_loss = val_loss / len(val_loader)

    print(f'Validation Loss: {average_val_loss}, Validation Accuracy: {val_accuracy}')

    # Save model to dictionary
    model_dict[f'noise_level_{int(100 * epsilon)}_sy'] = {
        'state_dict': model.state_dict(),
        'validation_loss': average_val_loss,
        'validation_accuracy': val_accuracy
    }


Symmetric Training with noise level: 0.1
Epoch 1/5, Loss: 1.7489376068115234
Epoch 2/5, Loss: 1.6000241041183472
Epoch 3/5, Loss: 1.5442806482315063
Epoch 4/5, Loss: 1.458237648010254
Epoch 5/5, Loss: 1.343650221824646
Validation Loss: 0.9062499176165101, Validation Accuracy: 0.7026
Symmetric Training with noise level: 0.3
Epoch 1/5, Loss: 2.3898043632507324
Epoch 2/5, Loss: 2.2614850997924805
Epoch 3/5, Loss: 2.120962142944336
Epoch 4/5, Loss: 2.144461154937744
Epoch 5/5, Loss: 1.9440466165542603
Validation Loss: 1.1075519858651859, Validation Accuracy: 0.6374
Symmetric Training with noise level: 0.5
Epoch 1/5, Loss: 2.1373746395111084
Epoch 2/5, Loss: 2.439639091491699
Epoch 3/5, Loss: 2.4658823013305664
Epoch 4/5, Loss: 2.3039183616638184
Epoch 5/5, Loss: 2.4154462814331055
Validation Loss: 1.2760925087959143, Validation Accuracy: 0.6011
Symmetric Training with noise level: 0.8
Epoch 1/5, Loss: 2.2473795413970947
Epoch 2/5, Loss: 2.276870012283325
Epoch 3/5, Loss: 2.299630641937256


In [25]:
# testing
for key, model_state_info in model_dict.items():

    model = BaselineModel()
    model_state = model.state_dict()
    model_state.update({k: v for k, v in model_state_info['state_dict'].items() if k in model_state})

    # Load the updated state_dict
    model.load_state_dict(model_state)

    model = model.to(device)
    model.eval()

    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)

            # Use the original CrossEntropyLoss without weighting for testing
            criterion = nn.CrossEntropyLoss()
            loss = criterion(outputs, labels)

            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = correct / total
    print(f'Test Accuracy for {key}: {accuracy}')


Test Accuracy for noise_level_10_sy: 0.7026
Test Accuracy for noise_level_30_sy: 0.6374
Test Accuracy for noise_level_50_sy: 0.6011
Test Accuracy for noise_level_80_sy: 0.3605
Test Accuracy for noise_level_90_sy: 0.0982


## Train & Validation and Testing - asymmetric noise label

In [ ]:
# noise_levels
noise_levels = [0.1, 0.3, 0.5, 0.8, 0.9]

# create a dictionary with keys in the format 'noise_level_{100 * value}'
model_dict = {f'noise_level_{int(100 * level)}_asy': None for level in noise_levels}

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Training loop with dynamic sample weighting
for epsilon in noise_levels:
    num_epochs = 5

    model = BaselineModel()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    model = model.to(device)
    model.train()

    print(f"Asymmetric Training with noise level: {epsilon}")

    for epoch in range(num_epochs):
        for inputs, labels in train_loader:

            # Add symmetric noise to labels
            labels = apply_label_noise(labels.numpy(), epsilon=epsilon, noise_type='asymmetric')
            labels = torch.from_numpy(labels)

            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()

            outputs = model(inputs)

            # Compute sample weights using the updated function
            weights = compute_sample_weights(outputs, labels)

            # Compute weighted loss
            loss = torch.mean(weights * criterion(outputs, labels))

            loss.backward()
            optimizer.step()

        print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item()}')

    # Validation
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)

            # Use the original CrossEntropyLoss without weighting for validation
            loss = criterion(outputs, labels)

            val_loss += loss.item()

            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_accuracy = correct / total
    average_val_loss = val_loss / len(val_loader)

    print(f'Validation Loss: {average_val_loss}, Validation Accuracy: {val_accuracy}')

    # Save model to dictionary
    model_dict[f'noise_level_{int(100 * epsilon)}_asy'] = {
        'state_dict': model.state_dict(),
        'validation_loss': average_val_loss,
        'validation_accuracy': val_accuracy
    }


In [ ]:
# testing
for key, model_state_info in model_dict.items():

    model = BaselineModel()
    model_state = model.state_dict()
    model_state.update({k: v for k, v in model_state_info['state_dict'].items() if k in model_state})

    # Load the updated state_dict
    model.load_state_dict(model_state)

    model = model.to(device)
    model.eval()

    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)

            # Use the original CrossEntropyLoss without weighting for testing
            criterion = nn.CrossEntropyLoss()
            loss = criterion(outputs, labels)

            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = correct / total
    print(f'Test Accuracy for {key}: {accuracy}')
